### python -m pip install pythonnet

In [1]:
from pathlib import Path
from enum import Enum
import json

In [2]:
data_folder = Path('./test_data/')
material_data = data_folder / 'material_data.json'
column_data = data_folder / 'column_data.json'
cross_section_data = data_folder / 'cross_section_data.json'
analytical_levels = data_folder / 'analytical_levels.json'


#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [3]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    print("No running instance of the program found or failed to attach.")

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)

N_mm_C = etabs.eUnits.N_mm_C
kN_m_C = etabs.eUnits.kN_m_C

display(myETABSObject)
display(SapModel)



# create stories

In [4]:
with open(analytical_levels, 'r') as file:
    levels = json.load(file)
level_dict = {k:v for k,v in levels}
sorted_level_dict = {k: v for k, v in sorted(level_dict.items(), key=lambda item: item[1])}
story_names = list(sorted_level_dict.keys())[1:]
story_heights = []
base_level = 0
for i,level in enumerate(sorted_level_dict.values()):
    if i == 0:
        base_level = level
        previous_level = level
        continue
    story_height = level - previous_level
    previous_level = level
    story_heights.append(story_height)

story_names
story_heights

def create_stories(SapModel,base_level:float,story_names:list[str],story_heights:list[float],unit = N_mm_C) -> tuple[int,list[str],list[float]]:
    ret = SapModel.SetPresentUnits(unit)
    stories_count = len(story_names)
    IsMasterStory = [False] * stories_count
    SimilarToStory = [''] * stories_count
    SpliceAbove = [False] * stories_count
    SpliceHeight = [0.0] * stories_count
    color = [0] * stories_count
    [ret, story_names, story_heights, IsMasterStory, SimilarToStory, SpliceAbove, SpliceHeight, color] = SapModel.Story.SetStories_2(
        base_level,
        stories_count,
        story_names,
        story_heights,
        IsMasterStory,
        SimilarToStory,
        SpliceAbove,
        SpliceHeight,
        color
    )
    return (ret, list(story_names), list(story_heights))

ret,_,_ = create_stories(SapModel,base_level,story_names,story_heights)


# create materials

In [5]:
with open(material_data, 'r') as file:
    materials = json.load(file)
materials

etabs_materials = {
    'concrete' : etabs.eMatType.Concrete,
    'steel': etabs.eMatType.Steel,
    'no_design' : etabs.eMatType.NoDesign
}


def get_etabs_concrete_material(name:str):
    return {
        'Name':name,
        'Fc':float(name[2:]),
        'IsLightweight':False,
        'FcsFactor':0.0,
        'SSType':2,
        'SSHysType':4,
        'StrainAtFc':0.0022,
        'StrainUltimate':0.005,
        'FinalSlope':-0.1,
        'FrictionAngle': 0.0,
        'DilatationalAngle' : 0.0,
        'Temp' : 0.0
    }


def get_etabs_steel_material(name:str):
    if name.upper() == 'S235':
        fy,fu,efy,efu = 235.0, 360.0, 258.5, 396.0
    elif name.upper() == 'S275':
        fy,fu,efy,efu = 275.0, 430.0, 302.5, 473.0
    elif name.upper() == 'S355':
        fy,fu,efy,efu = 355.0, 510.0, 390.5, 561.0
    elif name.upper() == 'S450':
        fy,fu,efy,efu = 440.0, 550.0, 484.0, 605.0
    elif name.upper() == 'A992FY50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A572GR50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A913GR50':
        fy,fu,efy,efu = 344.738, 413.685, 379.212, 455.054
    else:
        fy,fu,efy,efu = 248.211, 399.896, 372.317, 439.885

    return {
    'Name' : name,
    'Fy' : fy,
    'Fu' : fu,
    'EFy' : efy,
    'EFu' : efu,
    'SSType' : 1,
    'SSHysType' : 1,
    'StrainAtHardening' : 0.015,
    'StrainAtMaxStress' : 0.11,
    'StrainAtRupture' : 0.17,
    'FinalSlope' : -0.1,
    'Temp' : 0
    }



def add_material(SapModel, name:str,unit = N_mm_C):
    
    if name.upper().startswith('FC'):
        material_type = etabs_materials['concrete']
        fc = float(name[2:])
        E = 4700 * (fc)**0.5
        U = 0.2
        A = 0.0000099
        design_material = get_etabs_concrete_material(name)
    elif name.upper().startswith('S'):
        material_type = etabs_materials['steel']
        E = 199947.98
        U = 0.3
        A = 0.0000117
        design_material = get_etabs_steel_material(name)
    else:
        material_type = etabs_materials['no_design']
        E = 199947.98
        U = 0.3
        A = 0.0000117


    ret = SapModel.SetPresentUnits(unit)
    ret = SapModel.PropMaterial.SetMaterial(name, material_type)

    ret = SapModel.PropMaterial.SetMPIsotropic(
        name,
        E,
        U,
        A,
        Temp = 0
    )

    if material_type == etabs_materials['concrete']:
        ret = SapModel.PropMaterial.SetOConcrete_1(**design_material)
    elif material_type == etabs_materials['steel']:
        ret = SapModel.PropMaterial.SetOSteel_1(**design_material)


    ret = SapModel.SetPresentUnits(kN_m_C)
    weight = 25 if material_type == etabs_materials['concrete'] else 78.5
    ret = SapModel.PropMaterial.SetWeightAndMass(name, 1, weight)

    ret = SapModel.SetPresentUnits(unit)


_ = [add_material(SapModel,x) for x in materials]


# create cross sections

In [6]:
with open (cross_section_data, 'r') as file:
    sections = json.load(file)


In [7]:
def add_rebar_circular(section_name:str, rebars:str):
    rebar_count, rebar_size = rebars.upper().split('T')
    SetRebarColumn = {
        'Name': section_name,
        'MatPropLong':'A615Gr60',
        'MatPropConfine':'A615Gr60',
        'Pattern': 2,
        'ConfineType': 1,
        'Cover':40,
        'NumberCBars': int(rebar_count),
        'NumberR3Bars': 0,
        'NumberR2Bars': 0,
        'RebarSize': str(rebar_size),
        'TieSize':'10',
        'TieSpacingLongit':150,
        'Number2DirTieBars':3,
        'Number3DirTieBars':3,
        'ToBeDesigned': False
    }
    return SetRebarColumn

def add_rebar_rectangular(section_name:str, rebars:str,width_T2:float,depth_T3:float):
    rebar_count, rebar_size = rebars.upper().split('T')
    half_count_etabs = (int(rebar_count)-4) * 0.5 # without 4 bars at corner
    width_ratio = width_T2 / (width_T2 + depth_T3)
    NumberR3Bars = int(width_ratio * half_count_etabs)
    NumberR2Bars = max(int(half_count_etabs - NumberR3Bars),0) + 2
    NumberR3Bars += 2
    SetRebarColumn = {
        'Name': section_name,
        'MatPropLong':'A615Gr60',
        'MatPropConfine':'A615Gr60',
        'Pattern': 1,
        'ConfineType': 0,
        'Cover':40,
        'NumberCBars': 0,
        'NumberR3Bars': NumberR3Bars,
        'NumberR2Bars': NumberR2Bars,
        'RebarSize': str(rebar_size),
        'TieSize':'10',
        'TieSpacingLongit':150,
        'Number2DirTieBars':2,
        'Number3DirTieBars':4,
        'ToBeDesigned': False
    }
    return SetRebarColumn



In [8]:
# do not use this function , use the updated below
def get_etabs_column_concrete_section(section_data):
    size = str(section_data[1]['name']).split('_')[2]
    if str(section_data[1]['material_grade']).upper().startswith('FC'):
        if size.upper().startswith('D'):
            size = size[1:]
        name = f"{section_data[0]}-C {size}-{section_data[1]['rebar']}"
    else:
        name = f"{section_data[0]}-C {size}"
    return name

def get_etabs_column_concrete_section(section_data):
    size = str(section_data[1]['name']).split('_')[2]
    if str(section_data[1]['material_grade']).upper().startswith('FC'):
        if size.upper().startswith('D'):
            size = size[1:]
        name = f"C {size}-{section_data[1]['rebar']}-{section_data[1]['material_grade']}"
    else:
        name = f"C {size}"
    return name


def add_sections(SapModel, section_data, unit = N_mm_C):
    ret = SapModel.SetPresentUnits(unit)
    name = get_etabs_column_concrete_section(section_data)
    material:str = section_data[1]['material_grade']
    rebars = section_data[1]['rebar']
    if section_data[1]['h']:
        width_T2 = float(section_data[1]['b'])
        depth_T3 = float(section_data[1]['h'])
        #define rectangular frame section property
        ret = SapModel.PropFrame.SetRectangle(name, material, depth_T3, width_T2)
        if rebars:
            ret = SapModel.PropFrame.SetRebarColumn(**add_rebar_rectangular(name,rebars,width_T2, depth_T3))
    else:
        depth_T3 = float(section_data[1]['b'])
        ret = SapModel.PropFrame.SetCircle(name, material, depth_T3)
        if rebars:
            ret = SapModel.PropFrame.SetRebarColumn(**add_rebar_circular(name,rebars))
    if material.upper().startswith('FC'):
        #define frame section property modifiers
        ModValue = [1.0, 1.0, 1.0, 1.0, 0.7, 0.7, 1.0, 1.0]
        ret = SapModel.PropFrame.SetModifiers(name, ModValue)
    return name

etabs_section_names = [add_sections(SapModel,x) for x in sections]

etabs_section_names


['C 500x900-16T20-FC40',
 'C 900-24T20-FC30',
 'C 400x800-12T16-FC30',
 'C 500x900-18T20-FC40',
 'C 800x800-20T20-FC30',
 'C 450x450-8T16-FC30',
 'C 800x800-20T20-FC40',
 'C HE360A',
 'C SHS200x200x10',
 'C 500x900-None-FC40',
 'C 500x900-None-FC30']

# creating columns in etabs

In [9]:
with open (column_data, 'r') as file:
    columns = json.load(file)
columns = sorted(columns, key= lambda x : x[1]['z_base'])

# correct coordinates to follow column below
def update_columns_coordinates(columns):
    updated_columns = []
    for column in columns:
        id = column[0]
        column_below_id = column[1]['column_below']
        if column_below_id:
            updated_column = [id, {k:v for k,v in column[1].items()}]
            for column_below in columns:
                if column_below_id == column_below[0]:
                    updated_column[1]['x'] = column_below[1]['x']
                    updated_column[1]['y'] = column_below[1]['y']
                    updated_columns.append(updated_column)
                    break
        else:
            updated_columns.append(column)
    return updated_columns

sorted_columns = update_columns_coordinates(columns)
def get_column_coordinate(column):
    name = column[0]

    x = column[1]['x']
    y = column[1]['y']

    section_name = get_etabs_column_concrete_section(column)

    AddByCoord = {
        'XI':x,
        'YI':y,
        'ZI':column[1]['z_base'],
        'XJ':x,
        'YJ':y,
        'ZJ':column[1]['z_top'],
        'Name':str(name),
        'PropName' : section_name,
        'UserName' : str(name),
        'CSys' : 'Global'
    }
    return AddByCoord

def add_column(SapModel, column, all_columns):
    ret = SapModel.FrameObj.AddByCoord(**get_column_coordinate(column))
    column_name = str(column[0])
    angle = float(column[1]['angle'])
    # assign frame local axis angle
    ret = SapModel.FrameObj.SetLocalAxes(column_name, angle)


_ = [add_column(SapModel, x, sorted_columns) for x in sorted_columns]

# finish

In [10]:
SapModel = None
myETABSObject = None